<a href="https://colab.research.google.com/github/Bibhukalyan/chatbot_with_chat_history/blob/main/chatbots_remembering_chat_history.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv('GROQ_API_KEY')
model = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")

In [6]:
from langchain_core.messages import HumanMessage, AIMessage

model.invoke([
    HumanMessage(content="Hi, My name is bibhu i am a software engineer"),
    AIMessage(content="Hello Bibhu, it's nice to meet you! What can I do for you today?"),
    HumanMessage(content="What is my name and what i do?")
    ])

AIMessage(content="You are Bibhu, and you are a software engineer.  😊  \n\nIs there anything else you'd like to know or talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 60, 'total_tokens': 93, 'completion_time': 0.06, 'prompt_time': 0.001848854, 'queue_time': 0.012884732999999999, 'total_time': 0.061848854}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-eafcd1c5-3f48-463f-ad95-997626fb9711-0', usage_metadata={'input_tokens': 60, 'output_tokens': 33, 'total_tokens': 93})

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

session_store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
  if session_id not in session_store:
    session_store[session_id] = ChatMessageHistory()
  return session_store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [9]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke([ HumanMessage(content="Hi, My name is Bibhu i am a software engineer")], config=config)

In [13]:
print(response.content)

Hi Bibhu, it's great to meet you! 

What kind of software engineering do you do?  I'm curious to hear about your work. 😊  




In [14]:
response=with_message_history.invoke([ HumanMessage(content="What is my name and what i do")], config=config)

In [15]:
print(response.content)

You are Bibhu, and you are a software engineer!  😄 

Is there anything else you'd like to tell me about yourself or your work?  I'm happy to chat. 




In [16]:
##Changing the configuration
config1={"configurable":{"session_id":"chat2"}}

In [17]:
response=with_message_history.invoke([ HumanMessage(content="What is my name and what i do")], config=config1)
print(response.content)

As an AI, I have no access to personal information about you, including your name or what you do. If you'd like to tell me, I'm happy to learn! 😊  

What's your name and what do you do?



In [18]:
response=with_message_history.invoke([ HumanMessage(content="What is my name and what i do")], config=config)
print(response.content)

You said your name is Bibhu and that you are a software engineer!  

Do you want to tell me more about what kind of software engineering you do?  😊  



In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the question to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])
chain=prompt | model

In [20]:
result=chain.invoke({"messages":[HumanMessage(content="Hi, My name is Bibhu i am a software engineer")]})
result.content

"Hi Bibhu, it's nice to meet you! I'm glad to be your helpful assistant. \n\nWhat can I do for you today?  \n\nDo you have any questions I can answer or tasks I can help you with related to software engineering? I'm here to assist in any way I can. 😊  \n\n"

In [21]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)
##Changing the configuration
config3={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke([ HumanMessage(content="What is my name and what i do")], config=config3)
print(response.content)

As an AI, I have no memory of past conversations and don't know your name or what you do.

If you'd like to tell me, I'm happy to learn! 😊  




In [48]:
##adding complexity to prompt

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all the question to the best of your ability in {language}"),
    MessagesPlaceholder(variable_name="messages")
])
chain=prompt | model

with_message_history=RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
##Changing the configuration
config4={"configurable":{"session_id":"chat4"}}


In [51]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="what is my name")],
     "language":"Hindi"}, config=config4)
print(response.content)

आपका नाम Bibhukalyan है, जैसा कि आपने मुझे बताया था।  😊  



In [53]:
## Manage the coversation history
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are a helpful assistant. Answer all the question to the best of your ability"),
    HumanMessage(content="Hi, My name is Bibhu i am a software engineer"),
    AIMessage(content="Hello Bibhu, it's nice to meet you! What can I do for you today?"),
    HumanMessage(content="What is my name and what i do?")
    ]

print(trimmer.invoke(messages))

[SystemMessage(content='You are a helpful assistant. Answer all the question to the best of your ability', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi, My name is Bibhu i am a software engineer', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Bibhu, it's nice to meet you! What can I do for you today?", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is my name and what i do?', additional_kwargs={}, response_metadata={})]


In [54]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)

In [56]:
response=chain.invoke({"messages":messages + [HumanMessage(content="what is my name")],"language":"English"})
response.content

'Your name is Bibhu.  😊 \n\nYou told me earlier that you are a software engineer. \n'